# iSNV diversity calculations for H7N3 avian influenza virus genomes 
This Juptyer notebook was used to calculate iSNV diversity for avian influenza virus genomes sequenced from turkeys and chickens infected with H7N3 low or high pathogenicity avian viruses from the 2020 outbreak in North and South Carolina, United States. Its main goal is to calculate iSNV diversity metrics namely Shannon entropy and nucleotide diversity from variant data as determined by Lofreq. Lofreq outputs vcf files that contains the nucleotide change, position, and frequency of an identified iSNV. The vcf files are then converted into a dataframe from which iSNV diversity metrics are calculated from.

## Relevant references:

#### 1. Lofreq
##### Wilm A, Aw PP, Bertrand D, Yeo GH, Ong SH, Wong CH, Khor CC, Petric R, Hibberd ML, Nagarajan N. 2012. LoFreq: a sequence-quality aware, ultra-sensitive variant caller for uncovering cell-population heterogeneity from high-throughput sequencing datasets. Nucleic Acids Res 40:11189-201.

#### 2. Shannon entropy and Frequency cutoff of 2%
##### Fusaro A, Tassoni L, Milani A, Hughes J, Salviato A, Murcia PR, Massi P, Zamperin G, Bonfanti L, Marangon S, Cattoli G, Monne I. 2016. Unexpected Interfarm Transmission Dynamics during a Highly Pathogenic Avian Influenza Epidemic. J Virol 90:6401-11.
##### McCrone JT, Lauring AS. 2016. Measurements of Intrahost Viral Diversity Are Extremely Sensitive to Systematic Errors in Variant Calling. J Virol 90:6884-95.

#### 3. Nucleotide diversity
##### Zhao L, Illingworth CJR. 2019. Measurements of intrahost viral diversity require an unbiased diversity metric. Virus Evol 5.

In [1]:
import pandas as pd
import numpy as np
import os, math, re

wd='C:/Users/Christina.Leyson/OneDrive - USDA/Documents/Bioinformatics/1-H7N3 in turkeys and chickens/'

path_input_sample_key=wd+'sample_key_onesheet 3-29-2021.xlsx'
path_input_tables=wd+'MPJ-CL Plate 5.1-7.1/Lofreq tables/' #contains vcf files
path_input_grpvirusdose=wd+'grp_virus_dose_key.xlsx'

path_output_tables=wd+'MPJ-CL Plate 5.1-7.1/Lofreq tables/'
path_output_diversity=wd+'MPJ-CL Plate 5.1-7.1/Lofreq summary ntdiversity/iSNV_diversity.xlsx'
path_ouput_change_list=wd+'MPJ-CL Plate 5.1-7.1/Lofreq summary ntdiversity/SNV_list.xlsx'

cutoff=0.02
genome_length_dict={'HP':13608,'LP1':13581,'LP2':13515}

# 1. Create a sample key with information on the samples

In [2]:
#Create a function to define sample as either a swab or tissue sample
def smpl_class(row): 
    if row['sample_type'] in ['OP','CL']:
        return 'swab'
    elif row['sample_type'] in ['BR','HR','LU','MS','SP']:
        return 'tissue'

In [3]:
sample_key=pd.read_excel(path_input_sample_key)
sample_key['sample_class']=sample_key.apply(smpl_class, axis=1)

grpvirusdose_df=pd.read_excel(path_input_grpvirusdose)
sample_key['Dose']=sample_key['Group'].map(dict(zip(grpvirusdose_df['Group'],grpvirusdose_df['Dose'])))
sample_key.dtypes

Unnamed: 0                                             int64
SampleID                                              object
R1                                                     int64
R2                                                     int64
Pear                                                   int64
bam_wgs                                                int64
bam_wgs_fasta                                          int64
loreq                                                  int64
Library prep#                                        float64
Library plate                                        float64
Library prep position                                 object
RT-PCR date                                   datetime64[ns]
RT-PCR#                                              float64
RT-PCR Format                                         object
RT-PCR plate/strip#                                  float64
RT-PCR position                                       object
Opti-RT-PCR result      

In [4]:
virus_smplID_dict=pd.Series(sample_key.Virus.values,index=sample_key.SampleID).to_dict()

# 2. Reformat vcf file to tables

In [5]:
def vcf2excel(file,path_input):
    data=pd.read_csv(path_input+file,sep='\t',skiprows=18,
                     names=['CHROM','Position','ID','Reference','Sample','QUAL','FILTER','INFO'])
    INFO_list=data['INFO'].tolist()
    dp=[]
    af=[]
    sb=[]
    dp4=[]
    indel=[]
    hrun=[]
    for info in INFO_list:
        entry=info.split(';',6)
        dp.append((entry[0]).split('=')[1])
        af.append((entry[1]).split('=')[1])
        sb.append((entry[2]).split('=')[1])
        dp4.append((entry[3]).split('=')[1])
        if len(entry)>4:
            if entry[4] == 'INDEL':
                indel.append(True)
            else:
                indel.append(False)
    
            if 'HRUN' in entry[5]:
                hrun.append((entry[5]).split('=')[1])
            else:
                hrun.append('')
        else: 
            indel.append('')
            hrun.append('') 
    return data,dp,af,sb,dp4,indel,hrun

In [6]:
#Compile all data from vcf files into one dataframe
plates=['5.1','6','7.1','7.1.1']
for p in plates:
    path_input=wd+f'MPJ-CL Plate {p} - Galaxy/Lofreq Plate {p}/'
    file_list=os.listdir(path_input)
    for file in file_list:
        filename=(file.split('.'))[0]
        data,dp,af,sb,dp4,indel,hrun=vcf2excel(file,path_input)
        data['DP']=dp
        data['AF']=af
        data['SB']=sb
        data['DP4']=dp4
        data['INDEL']=indel
        data['HRUN']=hrun
        data['Segment']=data.CHROM.str.split('_').str[1]
        data['Segment_no']=data.Segment.str.split('-').str[0]
        data['Segment_name']=data.Segment.str.split('-').str[1]
        data['Change_name']=data['Segment']+'_'+data['Reference']+data['Position'].astype(str)+data['Sample']
        data['Segment_position']=data['Segment']+'_'+data['Position'].astype(str)
        data['SNV_test']=np.where((data.Sample.str.fullmatch('[ATCG]'))&(data.Reference.str.fullmatch('[ATCG]')),True,False)
        data.to_excel(path_output_tables+filename+'.xlsx')
print('Done.')
data.head()

Done.


,CHROM,Position,ID,Reference,Sample,QUAL,FILTER,INFO,DP,AF,SB,DP4,INDEL,HRUN,Segment,Segment_no,Segment_name,Change_name,Segment_position,SNV_test
0,LP1_8-NS,4,.,A,G,1676,PASS,"DP=221;AF=0.348416;SB=1;DP4=62,82,36,41",221,0.348416,1,"62,82,36,41",,,8-NS,8,NS,8-NS_A4G,8-NS_4,True
1,LP1_8-NS,337,.,T,A,1144,PASS,"DP=3747;AF=0.018148;SB=2;DP4=1796,1883,31,37",3747,0.018148,2,"1796,1883,31,37",,,8-NS,8,NS,8-NS_T337A,8-NS_337,True
2,LP1_8-NS,447,.,C,T,6294,PASS,"DP=3911;AF=0.070570;SB=3;DP4=1776,1857,142,134",3911,0.070570,3,"1776,1857,142,134",,,8-NS,8,NS,8-NS_C447T,8-NS_447,True
3,LP1_8-NS,611,.,T,C,90,PASS,"DP=3209;AF=0.002805;SB=4;DP4=1513,1687,6,3",3209,0.002805,4,"1513,1687,6,3",,,8-NS,8,NS,8-NS_T611C,8-NS_611,True
4,LP1_8-NS,622,.,A,G,62,PASS,"DP=3210;AF=0.002181;SB=1;DP4=1521,1681,4,3",3210,0.002181,1,"1521,1681,4,3",,,8-NS,8,NS,8-NS_A622G,8-NS_622,True


# 3. Determine how many iSNVs are present in each sample, average iSNV frequency (also allelic frequency, AF), Shannon entropy, and nucleotide diversity. Also get a list of unique SNVs across the whole dataset.

In [7]:
file_list_tables=os.listdir(path_input_tables)

change_list=[]

sampleID_list=[]
meanAF=[]
SNVcount=[]
entropy=[]

pi=[]

for file in file_list_tables:
    sampleID=file.split('_')[0]
    data=pd.read_excel(path_input_tables+file)
    
    #Get all SNVs that are above 2%
    data_cutoff=data.loc[(data['AF']>=cutoff)&(data['SNV_test']==True)]
    
    #Get a list of unique SNVs across the whole dataset
    changes_in_sample=data_cutoff['Change_name'].tolist()
    change_list=list(set(change_list+changes_in_sample))
    
    #Get mean allelic frequency AF and SNV count
    sampleID_list.append(sampleID)
    meanAF.append(data_cutoff['AF'].mean())
    SNVcount.append(data_cutoff['Change_name'].count())
    
    #Get a list of all loci/position where SNVs were found
    segpos_list=data_cutoff['Segment_position'].tolist()
    #print(len(data['Segment_position'].tolist()),len(segpos_list))
    
    #Calculate H is the sum of p*log p, p=frequency of the allele
    #See McCrone et al, JVI publication
    H_list=[]
    
    #Calculate Dl which is 1-sum of p^2
    #See Zhao and Illingworth, Virus Evolution publication
    Dl_list=[]
    
    for locus in segpos_list:
        locus_changes_df=data_cutoff.loc[data_cutoff['Segment_position']==locus]
        AF_list=locus_changes_df['AF'].tolist()
        
        #Get the frequency of the reference allele but if AF=1, no reference AF is added
        if 1 not in AF_list:     
            ref_AF=1-(sum(AF_list))
            AF_list.append(ref_AF)
        
        pisq_list=[]
        for AF in AF_list:
            #Calculated H for entropy
            H=(AF*(math.log10(AF)))
            H_list.append(H)
           
            pisq=AF**2
            pisq_list.append(pisq)
            #print(AF)
        #print(pisq_list, sum(pisq_list))
        
        #Calculate Dl for nucleotide diversity
        Dl=1-(sum(pisq_list))
        #print(Dl)
        Dl_list.append(Dl)
        #print(pisq_list,Dl)
    
    
    #Get virus genome length 
    virus=virus_smplID_dict.get(sampleID)
    genome_length=genome_length_dict.get(virus,13581)
    
    #Calculate Shannon entropy in the sample
    entropy_smpl=-(sum(H_list))/genome_length
    entropy.append(entropy_smpl)

    #Calculate nucleotide diversity pi in the sample
    #print(Dl_list,genome_length)
    pi_smpl=(sum(Dl_list))/genome_length
    pi.append(pi_smpl)
    
#Check methods :->
#print(sampleID,AF_list,data['Change_name'].count(),data_cutoff['Change_name'].count(),len(H_list),len(entropy),len(file_list_tables),len(pi))
#print(len(segpos_list),len(Dl_list)) 

In [8]:
diversity_calculations=pd.DataFrame({'SampleID':sampleID_list,
                                  'mean_AF':meanAF,
                                  'SNV_count':SNVcount,
                                  'Shannon_entropy':entropy,
                                  'Nucleotide_diversity':pi})

In [9]:
print('There a total of '+str(len(change_list))+' number of unique sites') 
change_list_df=pd.DataFrame({'Change_name':change_list})
change_list_df.to_excel(path_ouput_change_list)

There a total of 4656 number of unique sites


# 3. Combine sample information and iSNV diversity calculations

In [10]:
#Combine calculations and sample information
diversity_summary=pd.merge(sample_key,diversity_calculations,how='left',on='SampleID')
diversity_summary.to_excel(path_output_diversity)
diversity_summary.dtypes

Unnamed: 0                                             int64
SampleID                                              object
R1                                                     int64
R2                                                     int64
Pear                                                   int64
bam_wgs                                                int64
bam_wgs_fasta                                          int64
loreq                                                  int64
Library prep#                                        float64
Library plate                                        float64
Library prep position                                 object
RT-PCR date                                   datetime64[ns]
RT-PCR#                                              float64
RT-PCR Format                                         object
RT-PCR plate/strip#                                  float64
RT-PCR position                                       object
Opti-RT-PCR result      